In [4]:
%pip install --upgrade DBcm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\labuser\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import DBcm

In [6]:
print(dir(DBcm))

['ConnectionError', 'CredentialsError', 'SQLError', 'UseDatabase', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'mysql', 'sqlite3']


In [7]:
creds = {
    "host": "localhost",
    "database": "BlackjackDB",
    "user": "jack",
    "password": "jackpasswd",
}

In [8]:
creds

{'host': 'localhost',
 'database': 'BlackjackDB',
 'user': 'jack',
 'password': 'jackpasswd'}

In [9]:
with open("players.sql") as f:    # file context manager: "f" is a file alias.
    data = f.readlines()
# the file is automatically closed.

In [10]:
with DBcm.UseDatabase(creds) as db:    # database context manager: "db" is a database connection.
    SQL = "select * from games"
    db.execute(SQL)  # sends the SQL command to the server to run.
    results = db.fetchall()  # gets any results from the server.
# automatically closed the database connection.

In [11]:
results

[('Xx_TTV_MadJack69_xX', 'Loss'), ('Xx_TTV_MadJack69_xX', 'Loss')]

In [12]:
with DBcm.UseDatabase(creds) as db:    
    SQL = "select * from player"
    db.execute(SQL)  
    results = db.fetchall()  
results

[]

In [13]:
with DBcm.UseDatabase(creds) as db:    
    SQL = "describe player"
    db.execute(SQL)  
    results = db.fetchall()  
results

[('realname', 'varchar(32)', 'NO', '', None, ''),
 ('gamertag', 'varchar(32)', 'NO', '', None, ''),
 ('email', 'varchar(64)', 'NO', '', None, '')]

In [14]:
type(results)

list

In [15]:
type(results[0])

tuple

In [16]:
results[0][0]

'realname'

In [17]:
with DBcm.UseDatabase(creds) as db:    
    SQL = """
        select count(*) from games 
        where gamertag = "Xx_TTV_MadJack69_xX" and outcome = "Win"
    """
    db.execute(SQL)  
    results = db.fetchall()  
results

[(0,)]

In [18]:
with DBcm.UseDatabase(creds) as db:    
    SQL = """
        select count(*) from games 
        where gamertag = "Xx_TTV_MadJack69_xX" and outcome = "Win"
    """
    db.execute(SQL)  
    results = db.fetchone()  
results

(0,)

In [19]:
results[0]

0

In [20]:
with DBcm.UseDatabase(creds) as db:    
    SQL = """
        select count(*) from games 
        where gamertag = "Xx_TTV_MadJack69_xX" and outcome = "Win"
    """
    db.execute(SQL)  
    results = db.fetchone()  
    wins = results[0]
    SQL = """
        select count(*) from games 
        where gamertag = "Xx_TTV_MadJack69_xX" and outcome = "Loss"
    """
    db.execute(SQL)  
    results = db.fetchone() 
    losses = results[0]

print("Wins:", wins)
print("Losses:", losses)
print("Win percentage:", wins / (wins+losses))

Wins: 0
Losses: 2
Win percentage: 0.0


In [21]:
import random

In [22]:
SQL = """
    insert into games 
    ( gamertag, outcome )
    values     
    ("Xx_TTV_MadJack69_xX", "Loss")
"""
with DBcm.UseDatabase(creds) as db:
    db.execute(SQL)


In [23]:
outcome = random.choice(["Win", "Loss"])
outcome

'Loss'

In [24]:
with DBcm.UseDatabase(creds) as db:
    SQL = """
        select gamertag from player
    """
    db.execute(SQL)
    tags = db.fetchall()

In [25]:
print(tags)

[]


In [26]:
tags = [ row[0] for row in tags ]
print(tags)

[]


In [27]:
who = random.choice(tags)
who

IndexError: Cannot choose from an empty sequence

In [28]:
def random_gamer():
    with DBcm.UseDatabase(creds) as db:
        SQL = """
            select gamertag from player
        """
        db.execute(SQL)
        tags = db.fetchall()
        tags = [ row[0] for row in tags ]
        who = random.choice(tags)
    return who

In [29]:
gamer = random_gamer()
outcome = random.choice(["Win", "Loss"])
gamer, outcome

IndexError: Cannot choose from an empty sequence

In [30]:
for _ in range(50):
    SQL = """
        insert into games
        ( gamertag, outcome )
        values     
        (%s, %s)
    """
    gamer = random_gamer()
    outcome = random.choice(["Win", "Loss"])
    with DBcm.UseDatabase(creds) as db:
        db.execute(SQL, (gamer, outcome, ))   # tuple.
        SQL = """select * from games"""
        db.execute(SQL)
        results = db.fetchall()
        ## print(results[-3:])

IndexError: Cannot choose from an empty sequence

In [ ]:
len(results)

In [ ]:
results[25:50]

In [ ]:
with DBcm.UseDatabase(creds) as db:
    SQL = """select distinct gamertag from games"""
    db.execute(SQL)
    results = db.fetchall()
    print(len(results))

In [ ]:
for tag in tags:
    with DBcm.UseDatabase(creds) as db:    
        SQL = """
            select count(*) from games 
            where gamertag = %s and outcome = "Win"
        """
        db.execute(SQL, (tag, ))   # tuple.
        results = db.fetchone()  
        wins = results[0]
        SQL = """
            select count(*) from games 
            where gamertag = %s and outcome = "Loss"
        """
        db.execute(SQL, (tag, ))   # tuple.
        results = db.fetchone() 
        losses = results[0]

    ## print("Wins:", wins)
    ## print("Losses:", losses)
    print(tag, "win percentage:", wins / (wins+losses))
    print("-"*60)

In [ ]:
with DBcm.UseDatabase(creds) as db:
    SQL = """select gamertag from games"""
    db.execute(SQL)
    results = db.fetchall()
results[-20:]

In [ ]:
with DBcm.UseDatabase(creds) as db:
    SQL = """select gamertag from games
             order by gamertag desc"""
    db.execute(SQL)
    results = db.fetchall()
results[-20:]